Обучите модель Seq2Seq для перевода с английского на русский.

Для этого:

1. с сайта manythings.org выберите одну пару английских и русских предложений. Советуем выбрать ту пару предложений, которая не рассматривалась преподавателем на вебинаре.
2. используйте загруженный набор данных для обучения модели последовательности к последовательности (Seq2Seq) по аналогии с проведенным занятием. После завершения процесса обучения оцените качество работы модели.
3. добавьте еще один рекуррентный слой как в кодировщик (encoder), так и в декодировщик (decoder)
4. замените используемые вами GRU-ячейки на LSTM-ячейки. Повторно обучите модель и сравните результаты с предыдущими версиями.
5. сделайте выводы о качестве работы модели в каждом случае.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import random
import re

In [ ]:
english_sentences = [
    'What is your name?',
    'Where do you live?',
    'How old are you?',
    'I am from Russia.',
    'Do you speak English?'
]

russian_sentences = [
    'Как тебя зовут?',
    'Где ты живешь?',
    'Сколько тебе лет?',
    'Я из России.',
    'Ты говоришь по-английски?'
]

Создать словари и подготовить данные

In [ ]:
# Создать словари для английского и русского языков
SOS_token = 0
EOS_token = 1
UNK_token = 2

In [ ]:
def tokenize(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Zа-яА-Я0-9\s]", "", sentence)
    return sentence.split()

In [ ]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"<SOS>": SOS_token, "<EOS>": EOS_token, "<UNK>": UNK_token}
        self.index2word = {SOS_token: "<SOS>", EOS_token: "<EOS>", UNK_token: "<UNK>"}
        self.n_words = 3

    def add_sentence(self, sentence):
        for word in tokenize(sentence):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

    def indexes_from_sentence(self, sentence):
        return [self.word2index.get(word, self.word2index["<UNK>"]) for word in tokenize(sentence)] + [EOS_token]

In [ ]:
input_lang = Lang("English")
output_lang = Lang("Russian")

In [ ]:
def prepare_data(eng_snt, rus_snt):
    for eng, rus in zip(eng_snt, rus_snt):
        input_lang.add_sentence(eng)
        output_lang.add_sentence(rus)
    return [(input_lang.indexes_from_sentence(eng), output_lang.indexes_from_sentence(rus))
            for eng, rus in zip(eng_snt, rus_snt)]

In [ ]:
# Функция для преобразования пары предложений в тензоры
def tensors_from_pair(pair):
    input_tensor = torch.tensor(pair[0], dtype=torch.long).view(-1, 1)
    target_tensor = torch.tensor(pair[1], dtype=torch.long).view(-1, 1)
    return input_tensor, target_tensor

In [ ]:
pairs = prepare_data(english_sentences, russian_sentences)
print("Содержимое pairs:")
for i, pair in enumerate(pairs):
    print(f'Pair {i + 1}:')
    print(f'  Вход (English): {pair[0]}')
    print(f'  Выход (Russian): {pair[1]}')

Содержимое pairs:
Pair 1:
  Вход (English): [3, 4, 5, 6, 1]
  Выход (Russian): [3, 4, 5, 1]
Pair 2:
  Вход (English): [7, 8, 9, 10, 1]
  Выход (Russian): [6, 7, 8, 1]
Pair 3:
  Вход (English): [11, 12, 13, 9, 1]
  Выход (Russian): [9, 10, 11, 1]
Pair 4:
  Вход (English): [14, 15, 16, 17, 1]
  Выход (Russian): [12, 13, 14, 1]
Pair 5:
  Вход (English): [8, 9, 18, 19, 1]
  Выход (Russian): [7, 15, 16, 1]


In [ ]:
# Проверка соответствия индексов словам
print('\nСловарь input_lang (English):')
for idx, word in input_lang.index2word.items():
    print(f'  {idx}: {word}')

print('\nСловарь output_lang (Russian):')
for idx, word in output_lang.index2word.items():
    print(f'  {idx}: {word}')


Словарь input_lang (English):
  0: <SOS>
  1: <EOS>
  2: <UNK>
  3: what
  4: is
  5: your
  6: name
  7: where
  8: do
  9: you
  10: live
  11: how
  12: old
  13: are
  14: i
  15: am
  16: from
  17: russia
  18: speak
  19: english

Словарь output_lang (Russian):
  0: <SOS>
  1: <EOS>
  2: <UNK>
  3: как
  4: тебя
  5: зовут
  6: где
  7: ты
  8: живешь
  9: сколько
  10: тебе
  11: лет
  12: я
  13: из
  14: россии
  15: говоришь
  16: поанглийски


Создать модели и обучить их

In [ ]:
# Базовая модель
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [ ]:
# Базовая модель + дополнительный слой
class EncoderDeep(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderDeep, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(2, 1, self.hidden_size)

class DecoderDeep(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderDeep, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(2, 1, self.hidden_size)

In [ ]:
# Базовая модель + слои LSTM
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        input_tensor = torch.tensor(input_lang.indexes_from_sentence(sentence), dtype=torch.long)
        encoder_hidden = encoder.init_hidden()
        for i in range(input_tensor.size(0)):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
        decoder_input = torch.tensor([[SOS_token]])
        decoder_hidden = encoder_hidden
        decoded_words = []
        for _ in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [ ]:
def train(encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, n_epochs=1000, tf_ratio=0.5, eval_interval=100):
    for epoch in range(n_epochs):
        pair = random.choice(pairs)
        input_tensor, target_tensor = tensors_from_pair(pair)
        encoder_hidden = encoder.init_hidden()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)
        loss = 0

        # Прямой проход через кодировщик
        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]])
        decoder_hidden = encoder_hidden
        use_tf = True if random.random() < tf_ratio else False

        correct_words = 0
        total_words = 0

        if use_tf:
            for i in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                loss += criterion(decoder_output, target_tensor[i])
                decoder_input = target_tensor[i]

                topv, topi = decoder_output.topk(1)
                predicted_word = topi.squeeze().item()
                correct_word = target_tensor[i].item()
                if predicted_word == correct_word:
                    correct_words += 1
                total_words += 1
        else:
            for i in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                loss += criterion(decoder_output, target_tensor[i])
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()

                # Расчет точности
                predicted_word = topi.squeeze().item()
                correct_word = target_tensor[i].item()
                if predicted_word == correct_word:
                    correct_words += 1
                total_words += 1

        accuracy = correct_words / total_words if total_words > 0 else 0

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item() / target_length:.3f}, Accuracy: {accuracy:.3f}')

        if epoch % eval_interval == 0:
            print('--- Evaluation ---')
            test_sentence = random.choice(english_sentences)
            print(f'Данные: {test_sentence}')
            output_words = evaluate(encoder, decoder, test_sentence)
            output_sentence = ' '.join(output_words)
            print(f'Предсказание: {output_sentence}\n')

In [ ]:
# Базовая модель
encoder = Encoder(input_lang.n_words, 256)
decoder = Decoder(256, output_lang.n_words)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

train(encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, n_epochs=1000, eval_interval=200)

Epoch 0, Loss: 2.906, Accuracy: 0.000
--- Evaluation ---
Данные: Do you speak English?
Предсказание: ты говоришь говоришь говоришь говоришь говоришь говоришь говоришь говоришь говоришь

Epoch 100, Loss: 0.631, Accuracy: 1.000
Epoch 200, Loss: 0.103, Accuracy: 1.000
--- Evaluation ---
Данные: Do you speak English?
Предсказание: ты говоришь поанглийски <EOS>

Epoch 300, Loss: 0.057, Accuracy: 1.000
Epoch 400, Loss: 0.043, Accuracy: 1.000
--- Evaluation ---
Данные: What is your name?
Предсказание: как тебя зовут <EOS>

Epoch 500, Loss: 0.025, Accuracy: 1.000
Epoch 600, Loss: 0.027, Accuracy: 1.000
--- Evaluation ---
Данные: What is your name?
Предсказание: как тебя зовут <EOS>

Epoch 700, Loss: 0.015, Accuracy: 1.000
Epoch 800, Loss: 0.013, Accuracy: 1.000
--- Evaluation ---
Данные: Do you speak English?
Предсказание: ты говоришь поанглийски <EOS>

Epoch 900, Loss: 0.009, Accuracy: 1.000


**Выводы**:
Базовая модель быстро обучается и достигает высокой точности (1.000) уже к 200-й эпохе. Однако качество перевода в начале обучения низкое, что может быть связано с простотой архитектуры и ограниченностью данных.

In [ ]:
# Базовая модель + дополнительный слой
encoder_deep = EncoderDeep(input_lang.n_words, 256)
decoder_deep = DecoderDeep(256, output_lang.n_words)

encoder_optimizer = optim.SGD(encoder_deep.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder_deep.parameters(), lr=0.01)

train(encoder_deep, decoder_deep, encoder_optimizer, decoder_optimizer, criterion)

Epoch 0, Loss: 2.801, Accuracy: 0.000
--- Evaluation ---
Данные: Where do you live?
Предсказание: живешь тебя зовут тебя тебя тебя тебя зовут тебя тебя

Epoch 100, Loss: 1.203, Accuracy: 0.750
--- Evaluation ---
Данные: Do you speak English?
Предсказание: ты ты <EOS>

Epoch 200, Loss: 0.415, Accuracy: 1.000
--- Evaluation ---
Данные: How old are you?
Предсказание: сколько тебе лет <EOS>

Epoch 300, Loss: 0.211, Accuracy: 1.000
--- Evaluation ---
Данные: What is your name?
Предсказание: как тебя зовут <EOS>

Epoch 400, Loss: 0.067, Accuracy: 1.000
--- Evaluation ---
Данные: Do you speak English?
Предсказание: ты говоришь поанглийски <EOS>

Epoch 500, Loss: 0.088, Accuracy: 1.000
--- Evaluation ---
Данные: Do you speak English?
Предсказание: ты говоришь поанглийски <EOS>

Epoch 600, Loss: 0.039, Accuracy: 1.000
--- Evaluation ---
Данные: How old are you?
Предсказание: сколько тебе лет <EOS>

Epoch 700, Loss: 0.036, Accuracy: 1.000
--- Evaluation ---
Данные: Do you speak English?
Предсказ

**Выводы**: Добавление дополнительного слоя улучшает способность модели обрабатывать сложные зависимости, но замедляет процесс обучения. Начальная точность выше, чем у базовой модели, что указывает на лучшую способность захватывать контекст. Однако время, необходимое для достижения точности 1.000, увеличивается.

In [ ]:
# Базовая модель + слои LSTM
encoder_lstm = EncoderLSTM(input_lang.n_words, 256)
decoder_lstm = DecoderLSTM(256, output_lang.n_words)

encoder_optimizer = optim.SGD(encoder_lstm.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder_lstm.parameters(), lr=0.01)

train(encoder_lstm, decoder_lstm, encoder_optimizer, decoder_optimizer, criterion)

Epoch 0, Loss: 2.816, Accuracy: 0.000
--- Evaluation ---
Данные: What is your name?
Предсказание: я <EOS>

Epoch 100, Loss: 1.913, Accuracy: 0.250
--- Evaluation ---
Данные: How old are you?
Предсказание: ты говоришь поанглийски <EOS>

Epoch 200, Loss: 2.203, Accuracy: 0.250
--- Evaluation ---
Данные: Where do you live?
Предсказание: где ты живешь <EOS>

Epoch 300, Loss: 0.323, Accuracy: 1.000
--- Evaluation ---
Данные: How old are you?
Предсказание: сколько тебе лет <EOS>

Epoch 400, Loss: 0.078, Accuracy: 1.000
--- Evaluation ---
Данные: I am from Russia.
Предсказание: я из россии <EOS>

Epoch 500, Loss: 0.062, Accuracy: 1.000
--- Evaluation ---
Данные: How old are you?
Предсказание: сколько тебе лет <EOS>

Epoch 600, Loss: 0.032, Accuracy: 1.000
--- Evaluation ---
Данные: I am from Russia.
Предсказание: я из россии <EOS>

Epoch 700, Loss: 0.025, Accuracy: 1.000
--- Evaluation ---
Данные: Where do you live?
Предсказание: где ты живешь <EOS>

Epoch 800, Loss: 0.029, Accuracy: 1.000
--

**Выводы**: Модель с LSTM демонстрирует самое медленное обучение в начале, но затем быстро догоняет другие модели. После 300 эпох она достигает точности 1.000, что указывает на высокое качество перевода.

**Общие выводы:**
* Базовая модель обучается быстрее всех, но её простота может ограничивать её способность обрабатывать сложные зависимости.
* Добавление дополнительного слоя улучшает способность модели обобщать данные, но замедляет обучение.
* Увеличение глубины сети или использование LSTM повышает качество перевода, но также увеличивает вычислительные затраты.